In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings 
# warnings.filterwarnings('ignore')

In [4]:
import kagglehub
import os

# Download latest version
path = kagglehub.dataset_download("bhadramohit/customer-shopping-latest-trends-dataset")

print("Path to dataset files:", path)

Path to dataset files: /home/dude/.cache/kagglehub/datasets/bhadramohit/customer-shopping-latest-trends-dataset/versions/1


In [5]:
os.listdir(path)

['shopping_trends.csv']

In [15]:
df = pd.read_csv(path + '/shopping_trends.csv')
df.drop('Customer ID', axis=1, inplace=True)

In [16]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [17]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3120 entries, 247 to 3174
Data columns (total 18 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       3120 non-null   int64  
 1   Gender                    3120 non-null   object 
 2   Item Purchased            3120 non-null   object 
 3   Category                  3120 non-null   object 
 4   Purchase Amount (USD)     3120 non-null   int64  
 5   Location                  3120 non-null   object 
 6   Size                      3120 non-null   object 
 7   Color                     3120 non-null   object 
 8   Season                    3120 non-null   object 
 9   Review Rating             3120 non-null   float64
 10  Subscription Status       3120 non-null   object 
 11  Payment Method            3120 non-null   object 
 12  Shipping Type             3120 non-null   object 
 13  Discount Applied          3120 non-null   object 
 14  Promo Code 

In [18]:
train_df.head()

,Age,Gender,Item Purchased,Category,Purchase Amount (USD),Location,Size,Color,Season,Review Rating,Subscription Status,Payment Method,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Preferred Payment Method,Frequency of Purchases
247,29,Male,Sweater,Clothing,66,Montana,XL,Brown,Spring,4.4,Yes,Credit Card,Standard,Yes,Yes,14,Debit Card,Weekly
2591,40,Male,Jacket,Outerwear,61,New Mexico,S,Olive,Spring,3.3,No,Debit Card,Express,No,No,40,Cash,Quarterly
1822,66,Male,Shirt,Clothing,85,Connecticut,S,Cyan,Summer,4.4,No,Cash,Standard,No,No,4,Cash,Annually
3483,19,Female,Hoodie,Clothing,37,Texas,L,White,Winter,3.9,No,Venmo,Express,No,No,21,Credit Card,Every 3 Months
2771,58,Female,Jacket,Outerwear,52,Maryland,S,Beige,Summer,3.1,No,Cash,2-Day Shipping,No,No,12,Bank Transfer,Fortnightly
